In [ ]:
import arcpy, os

#import 
model_inputs_TIFF30m_path = r"D:\Dropbox\x\PostDoc\23 10 LMU\13 exp_cases\24 01 Urban SSP MAN MUM JAK\05_GIS_temp\MAN\model_inputs\TIFF_30m"
model_inputs_TIFF150m_path = r"D:\Dropbox\x\PostDoc\23 10 LMU\13 exp_cases\24 01 Urban SSP MAN MUM JAK\05_GIS_temp\MAN\model_inputs\TIFF_150m"
model_inputs_ASCII150m_path = r"D:\Dropbox\x\PostDoc\ASCII_150m"
case_city = 'MAN'
real_estate_file = '_ECO_real_estate_EBK_Lamuidi_reclass_2024_30m.tif'

arcpy.env.workspace = model_inputs_TIFF30m_path
resolution = 150
nn_list  = [case_city + real_estate_file,
            case_city + '_LIM_districts_GADM4_30m.tif',
            case_city + '_LIM_exclusion_layer_30m.tif',
            case_city + '_POP_census_income_2015_30m.tif',
            case_city + '_TRA_roads_OSM_2024_L_all_30m.tif',
            case_city + '_URB_tenure_2016_MMRDA_30m.tif', 
            case_city + '_URB_urbanisation_1985_EOC_WUF_30m.tif', 
            case_city + '_URB_urbanisation_2000_EOC_WUF_30m.tif', 
            case_city + '_URB_urbanisation_2015_EOC_WUF_30m.tif']
cubic_list = [case_city + '_DEM_slope_pct_30m.tif', 
              case_city + '_LIM_districts_GADM4_30m.tif',
              case_city + '_LOC_airports_OSM_2024_distance_normal_30m.tif', 
              case_city + '_LOC_CBDs_OSM_2024_distance_normal_30m.tif', 
              case_city + '_LOC_health_OSM_2024_distance_normal_30m.tif', 
              case_city + '_LOC_malls_OSM_2024_distance_normal_30m.tif', 
              case_city + '_LOC_markets_OSM_2024_distance_normal_30m.tif', 
              case_city + '_LOC_schools_OSM_2024_distance_normal_30m.tif', 
              case_city + '_LOC_suburban_centres_OSM_2024_distance_normal_30m.tif',
              case_city + '_POP_density_normal_2000_WDPop_30m.tif', 
              case_city + '_POP_density_normal_2015_WDPop_30m.tif', 
              case_city + '_TRA_attractive_areas_OSM_2024_P_distance_normal_30m.tif',
              case_city + '_TRA_distance_to_major_roads_30m.tif', 
              case_city + '_URB_distance_urbanisation_2000_EOC_WUF_30m.tif', 
              case_city + '_URB_distance_urbanisation_2015_EOC_WUF_30m.tif']
raster_list = list(arcpy.ListRasters ("*"+case_city+"_*", "TIF"))
print('There are ',len(raster_list),' rasters in the 30m input folder.\n And a total of ',
      len(nn_list)+len(cubic_list),' in the lists (',len(nn_list),'NN & ',len(cubic_list),'cubic)')
for r in raster_list:
    print(r)

    

In [ ]:
# resample rasters in the cubic_list using resampling_type="CUBIC" and those in nn_list using  resampling_type="NEAREST" 
out_folder = model_inputs_TIFF150m_path
snap_raster = case_city + "_LIM_raster_reference_150m.tif"

with arcpy.EnvManager(snapRaster=snap_raster):
    for r in arcpy.ListRasters ("*"+case_city+"_*", "TIF"): #"*"+case_city+"_*", "TIF"
        out_file = r.replace('_30m','_{0}m'.format(resolution))
        out_raster = os.path.join(out_folder, out_file)
        #print(out_raster)
        if r in nn_list:
            try:
                arcpy.Resample_management(in_raster=r, out_raster=out_raster, cell_size=str(resolution), resampling_type="NEAREST")
                print(r,'succeeded')
            except:
                print('**Error with',r)
        elif r in cubic_list:
            try:
                arcpy.Resample_management(in_raster=r, out_raster=out_raster, cell_size=str(resolution), resampling_type="CUBIC")
                print(r,'succeeded')
            except:
                print('**Error with',r)
            #normal_list = normal_list.append(r)
print("Finished.")

In [ ]:
arcpy.env.workspace = model_inputs_TIFF150m_path
raster_150_list = list(arcpy.ListRasters ("*150m*", "TIF"))
print('the number of rasters is', len(raster_150_list),'\n')
for r in raster_150_list:
    print(r)

In [ ]:
out_folder =  model_inputs_ASCII150m_path # the folder path cannot have spaces in it
arcpy.env.workspace = model_inputs_TIFF150m_path

resolution = 150

with arcpy.EnvManager(snapRaster="MAN_LIM_raster_reference_150m.tif"):
     for r in arcpy.ListRasters ("*150m*", "TIF"): #MAN_POP_
            out_ascii_file = os.path.join(out_folder, r.replace('.tif','.asc'))
            print('TIFF file:',r,
                  '\nASCI file:',out_ascii_file)
            try:
                arcpy.conversion.RasterToASCII(r, out_ascii_file)
            except:
                print('****Failed****, skipping.')
      
    
     print('Finished.')